In [72]:
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go

In [93]:
file_path = "C:/Users/maria/Documents/Documentos Pessoais/MBA/Mestrado/saude-basica/saude-basica/painel/data/atendimentos.json"
with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)


In [140]:
# Mapeamento dos meses para seus números correspondentes
mes_map = {
    'JAN': 1, 'FEV': 2, 'MAR': 3, 'ABR': 4, 'MAI': 5, 'JUN': 6,
    'JUL': 7, 'AGO': 8, 'SET': 9, 'OUT': 10, 'NOV': 11, 'DEZ': 12
}
# Mapeamento dos meses para seus trimestres correspondentes
trimestre_map = {
    'JAN': 'T1', 'FEV': 'T1', 'MAR': 'T1', 'ABR': 'T2', 'MAI': 'T2', 'JUN': 'T2',
    'JUL': 'T3', 'AGO': 'T3', 'SET': 'T3', 'OUT': 'T4', 'NOV': 'T4', 'DEZ': 'T4'
}


In [142]:
# Convertendo os dados em um DataFrame
dados = []

# Iterar sobre os diferentes tipos (enfermeiro, médico, etc.)
for tipo, anos in json_data.items():
    # Iterar sobre os anos (2013, 2014, etc.)
    for ano, meses in anos.items():
        # Iterar sobre os meses e seus valores
        for mes, valor in meses.items():
            # Adicionar uma linha para cada entrada
            dados.append([tipo, int(ano), trimestre_map[mes], mes_map[mes], valor])

# Criar o DataFrame com as colunas: tipo, ano, mês, valor
df = pd.DataFrame(dados, columns=['tipo', 'ano', 'trimestre', 'mes', 'valor'])
# df['trimestre'] = df['mes'].apply(calcular_trimestre)
df['ano_mes'] = df['mes'].astype(str) + '/' + df['ano'].astype(str)
df['ano_trimestre'] = df['trimestre'].astype(str) + '/' + df['ano'].astype(str)

In [143]:
# Criação do gráfico de barras empilhadas usando Plotly Express
fig = px.bar(
    df, 
    x='ano_mes', 
    y='valor', 
    color='tipo', 
    title="Atendimentos por Ano/Mês e Tipo", 
    labels={'ano_mes': 'Ano/Mês', 'valor': 'Valor'},
    barmode='stack'  # Definir como empilhado
)


In [144]:
fig.show()

In [103]:
# Gráfico dos últimos 6 trimestres

# Agrupar os dados por ano e quarter somando os valores
df_grouped = df.groupby(['ano_trimestre', 'ano', 'trimestre'], observed=True)['valor'].sum().reset_index()

# Ordenar por ano e quarter para garantir o filtro correto
df_grouped['ano_order'] = df_grouped['ano'].astype(str) + df_grouped['trimestre'].astype(str).str.replace('T', '')  
df_grouped = df_grouped.sort_values('ano_order')

# Filtrar os últimos 6 trimestres
df_filtered = df_grouped.tail(6)

# Criar gráfico de barras
quarter_chart = px.bar(
    df_filtered, 
    x="ano_trimestre", 
    y="valor", 
    text_auto=".2s",
    title="Atendimentos por Trimestre"
)

# Atualizar as trace e o hovertemplate
quarter_chart.update_traces(
    textposition="outside",
    marker_color="#f79500",
    hoverlabel=dict(bgcolor="rgba(255, 255, 255, 0.1)", font_size=12),
    hovertemplate="<b>%{y:,.0f}</b><br>atendimentos em %{x}<extra></extra>"
)

# Atualizar layout do gráfico
quarter_chart.update_layout(
    xaxis_title=None,
    yaxis_title=None,
    plot_bgcolor="rgba(0, 0, 0, 0)",
    yaxis=dict(showticklabels=False),
    margin=dict(l=35, r=35, t=60, b=40),
)
      
# Exibir gráfico
quarter_chart.show()


In [106]:
# Gráfico dos últimos 6 anos

# Agrupar os dados por ano e quarter somando os valores
df_grouped = df.groupby(['ano'], observed=True)['valor'].sum().reset_index()

df_grouped = df_grouped.sort_values('ano')

# Filtrar os últimos 6 trimestres
df_filtered = df_grouped.tail(6)

# Criar gráfico de barras
ano_chart = px.bar(
    df_filtered, 
    x="ano", 
    y="valor", 
    text_auto=".2s",
    title="Atendimentos por Ano"
)

# Atualizar as trace e o hovertemplate
ano_chart.update_traces(
    textposition="outside",
    marker_color="#632956",
    hoverlabel=dict(bgcolor="#343A40", font_size=12),
    hovertemplate="<b>%{y:,.0f}</b><br>atendimentos em %{x}<extra></extra>"
)

# Atualizar layout do gráfico
ano_chart.update_layout(
    xaxis_title=None,
    yaxis_title=None,
    plot_bgcolor="rgba(0, 0, 0, 0)",
    yaxis=dict(showticklabels=False),
    margin=dict(l=35, r=35, t=60, b=40),
)

# Exibir gráfico
ano_chart.show()


----------------------------------------------------------------------------------------------------------------------------

CONSIDERAR SÓ OS CÓDIGOS A SEGUIR

-------------------------------------------------------------------------------------------------------------

TRANSFORMA JSON EM DF ADAPTADO PARA CONSIDERAR NA GERAÇÃO DOS GRÁFICOS

In [146]:
# Mapeamento dos meses para seus números correspondentes
mes_map = {
    'JAN': 1, 'FEV': 2, 'MAR': 3, 'ABR': 4, 'MAI': 5, 'JUN': 6,
    'JUL': 7, 'AGO': 8, 'SET': 9, 'OUT': 10, 'NOV': 11, 'DEZ': 12
}
# Mapeamento dos meses para seus trimestres correspondentes
trimestre_map = {
    'JAN': 'T1', 'FEV': 'T1', 'MAR': 'T1', 'ABR': 'T2', 'MAI': 'T2', 'JUN': 'T2',
    'JUL': 'T3', 'AGO': 'T3', 'SET': 'T3', 'OUT': 'T4', 'NOV': 'T4', 'DEZ': 'T4'
}


In [189]:
def get_df_atendimentos(json_data):
    # Para transformar um json de atendimento em um df que será utilizado para gerar os gráficos
    #    json_data -> json que contem os dados de atendimento 
    # retorna o df
    dados = []

    # Iterar sobre os diferentes tipos (enfermeiro, médico, etc.)
    for profissional, anos in json_data.items():
        # Iterar sobre os anos (2013, 2014, etc.)
        for ano, meses in anos.items():
            # Iterar sobre os meses e seus valores
            for mes, valor in meses.items():
                # Adicionar uma linha para cada entrada
                dados.append([profissional, int(ano), trimestre_map[mes], mes_map[mes], valor])

    # Criar o DataFrame com as colunas: profissional, ano, mês, valor
    df = pd.DataFrame(dados, columns=['profissional', 'ano', 'trimestre', 'mes', 'valor'])
    # df['trimestre'] = df['mes'].apply(calcular_trimestre)
    df['ano_mes'] = df['mes'].astype(str) + '/' + df['ano'].astype(str)
    df['ano_trimestre'] = df['trimestre'].astype(str) + '/' + df['ano'].astype(str)

    #TODO normalizar valores pelo total da população (1000 habitantes)

    return df

In [186]:
def get_df_altas(json_data):
    # Para transformar um json de atendimento em um df que será utilizado para gerar os gráficos
    #    json_data -> json que contem os dados de atendimento 
    # retorna o df
    dados = []

    # Iterar sobre os anos (2013, 2014, etc.)
    for ano, meses in json_data.items():
        # Iterar sobre os meses e seus valores
        for mes, valor in meses.items():
            # Adicionar uma linha para cada entrada
            dados.append([int(ano), trimestre_map[mes], mes_map[mes], valor])

    # Criar o DataFrame com as colunas: tipo, ano, mês, valor
    df = pd.DataFrame(dados, columns=['ano', 'trimestre', 'mes', 'valor'])
    # df['trimestre'] = df['mes'].apply(calcular_trimestre)
    df['ano_mes'] = df['mes'].astype(str) + '/' + df['ano'].astype(str)
    df['ano_trimestre'] = df['trimestre'].astype(str) + '/' + df['ano'].astype(str)

    #TODO normalizar valores pelo total da população (1000 habitantes)
    
    return df

Teste

In [190]:
file_path = "C:/Users/maria/Documents/Documentos Pessoais/MBA/Mestrado/saude-basica/saude-basica/painel/data/atendimentos.json"
with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)
df = get_df_atendimentos(json_data)
print(df)

    profissional   ano trimestre  mes    valor  ano_mes ano_trimestre
0     enfermeiro  2013        T2    4     1061   4/2013       T2/2013
1     enfermeiro  2013        T3    8     6763   8/2013       T3/2013
2     enfermeiro  2013        T4   12    17126  12/2013       T4/2013
3     enfermeiro  2013        T3    7     4386   7/2013       T3/2013
4     enfermeiro  2013        T2    6      937   6/2013       T2/2013
..           ...   ...       ...  ...      ...      ...           ...
403       outros  2024        T1    1  1760250   1/2024       T1/2024
404       outros  2024        T3    7  2297005   7/2024       T3/2024
405       outros  2024        T2    6  2220100   6/2024       T2/2024
406       outros  2024        T2    5  2141932   5/2024       T2/2024
407       outros  2024        T1    3  2036724   3/2024       T1/2024

[408 rows x 7 columns]


BIG NUMBERS

In [191]:
def get_big_numbers_atendimentos(df, ano):
    # Para obter os big numbers que devem aparecer como resumo
    #    df -> dados para gerar o gráfico que deve conter ['tipo', 'ano']
    #    ano -> int, ano que foi selecionado para trazer os dados
    # retorna lista com o total do ano e abertura por médico, enfermeiro e outros
    total_ano = df[df['ano'] == ano]['valor'].sum()
    medico = df[(df['profissional'] == 'medico') & (df['ano'] == ano)]['valor'].sum()
    enfermeiro = df[(df['profissional'] == 'enfermeiro') & (df['ano'] == ano)]['valor'].sum()
    outros = df[(df['profissional'] == 'outros') & (df['ano'] == ano)]['valor'].sum()

    return [total_ano, medico, enfermeiro, outros]

Teste

In [192]:
get_big_numbers_atendimentos(df, 2023)

[np.int64(337097074),
 np.int64(206604444),
 np.int64(107725663),
 np.int64(22766967)]

GRÁFICOS

In [115]:
# Mapeamento das cores para cada um dos tipos de gráficos [claro, medio, escuro]
type_color_map = {
    'brasil': ["#B36CA3" ,"#632956", "#3B032F"],
    'estado': ["#80B0DC" ,"#34679A", "#11173F"],
    'regiao_saude': ["#97C471" ,"#2B7B6F", "#11302B"],
    'municipio': ["#FFC20D" ,"#F7941C", "#A7620E"]
}

In [214]:
def update_layout_chart(chart, title, type):
    # Para atualizar o layout do gráfico
    #    chart -> o gráfico que vamos alterar
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico atualizado
    
    color = type_color_map.get(type, [None])[1]

    chart.update_traces(
        textposition="outside",
        marker_color=color,
        hoverlabel=dict(bgcolor='#FFFFFF', font_color="#343A40", font_size=12),
        hovertemplate=f"<b>%{{y:,.0f}}</b><br>{title} em %{{x}}<extra></extra>"
    )

    chart.update_layout(
        xaxis_title=None,
        yaxis_title=None,
        plot_bgcolor="#FFFFFF",
        yaxis=dict(showticklabels=False),
        margin=dict(l=35, r=35, t=60, b=40),
    )

    return chart

In [234]:
def update_layout_chart_profissionais(chart, title, type):
    # Para atualizar o layout do gráfico
    #    chart -> o gráfico que vamos alterar
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico atualizado
    
    color1 = type_color_map.get(type, [None])[0]
    color2 = type_color_map.get(type, [None])[1]

    # Aplicar as duas cores alternadamente
    chart.update_traces(
        textposition="outside",
        hoverlabel=dict(bgcolor='#FFFFFF', font_color="#343A40", font_size=12),
        hovertemplate=f"<b>%{{y:,.0f}}</b><br>{title} por %{{fullData.name}} em %{{x}}<extra></extra>"
    )

    chart.update_layout(
        xaxis_title=None,
        yaxis_title=None,
        legend_title=None,
        plot_bgcolor="#FFFFFF",
        yaxis=dict(showticklabels=False),
        margin=dict(l=35, r=35, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1) #, xanchor="center", x=0.5 )
    )

    return chart

In [235]:
def get_chart_by_year_profissionais(df, title, type):
    # Retorna o gráfico de barras com o total acumulado dos últimos 6 anos de dados
    #    df -> dados para gerar o gráfico que deve conter ['ano', 'valor']
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico gerado
    
    # Agrupar os dados por ano e quarter somando os valores
    df = df[(df['profissional'] == 'medico') | (df['profissional'] == 'enfermeiro')]
    df_grouped = df.groupby(['profissional', 'ano'], observed=True)['valor'].sum().reset_index()
    df_grouped = df_grouped.sort_values('ano')
    df_filtered = df_grouped.tail(6 * df_grouped['profissional'].nunique())

    color1 = type_color_map.get(type, [None])[0]
    color2 = type_color_map.get(type, [None])[1]

    # Criar gráfico de barras empilhadas
    chart = px.bar(
        df_filtered, 
        x="ano", 
        y="valor", 
        color="profissional",  # Agrupar por profissional
        text_auto=".2s",
        title=f"{title.capitalize()} por Ano",
        labels={"ano": "Ano", "valor": "Valor", "profissional": "Profissional"},
        color_discrete_map={  # Mapear cores específicas para cada profissional
        'medico': color1,
        'enfermeiro': color2
    }
    )

    # Atualizar para o layout padrão
    chart = update_layout_chart_profissionais(chart, title, type)

    return chart

In [197]:
def get_chart_by_year(df, title, type):
    # Retorna o gráfico de barras com o total acumulado dos últimos 6 anos de dados
    #    df -> dados para gerar o gráfico que deve conter ['ano', 'valor']
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico gerado
    
    # Agrupar os dados por ano e quarter somando os valores
    df_grouped = df.groupby(['ano'], observed=True)['valor'].sum().reset_index()
    df_grouped = df_grouped.sort_values('ano')
    df_filtered = df_grouped.tail(6)

    # Criar gráfico de barras
    chart = px.bar(
        df_filtered, 
        x="ano", 
        y="valor", 
        text_auto=".2s",
        title=f"{title.capitalize()} por Ano"
    )

    # Atualizar para o layout padrão
    chart = update_layout_chart(chart, title, type)

    return chart

In [137]:
def get_chart_by_quarter(df, title, type):
    # Retorna o gráfico de barras com o total acumulado dos últimos 6 trimestres de dados
    #    df -> dados para gerar o gráfico que deve conter ['ano_trimestre', 'ano', 'trimestre', 'valor']
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico gerado

    # Agrupar os dados por ano e quarter somando os valores
    df_grouped = df.groupby(['ano_trimestre', 'ano', 'trimestre'], observed=True)['valor'].sum().reset_index()
    df_grouped['ano_order'] = df_grouped['ano'].astype(str) + df_grouped['trimestre'].astype(str).str.replace('T', '')  
    df_grouped = df_grouped.sort_values('ano_order')
    df_filtered = df_grouped.tail(6)

    # Criar gráfico de barras
    chart = px.bar(
        df_filtered, 
        x="ano_trimestre", 
        y="valor", 
        text_auto=".2s",
        title=f"{title.capitalize()} por Trimestre"
    )

    # Atualizar para o layout padrão
    chart = update_layout_chart(chart, title, type)

    return chart

In [135]:
def get_chart_by_month(df, title, type):
    # Retorna o gráfico de barras com o total acumulado dos últimos 6 trimestres de dados
    #    df -> dados para gerar o gráfico que deve conter ['ano_mes', 'ano', 'mes', 'valor']
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico gerado

    # Agrupar os dados por ano e mes somando os valores
    df_grouped = df.groupby(['ano_mes', 'ano', 'mes'], observed=True)['valor'].sum().reset_index()
    df_grouped['ano_order'] = df_grouped['ano'].astype(str) + df_grouped['mes'].astype(str)
    df_grouped = df_grouped.sort_values('ano_order')
    df_filtered = df_grouped.tail(18)


    # Criar gráfico de barras
    chart = px.bar(
        df_filtered, 
        x="ano_mes", 
        y="valor", 
        text_auto=".2s",
        title=f"{title.capitalize()} por Mês"
    )

    # Atualizar para o layout padrão
    chart = update_layout_chart(chart, title, type)

    return chart

Testes

In [236]:
get_chart_by_year_profissionais(df, "atendimentos", "municipio")

In [139]:
get_chart_by_quarter(df, "atendimentos", "estado")

In [145]:
get_chart_by_quarter(df, "atendimentos por 100M hab", "brasil")

In [136]:
#teste 
get_chart_by_month(df, "atendimentos", "municipio")